In [1]:
import os
import sys
import ast
import json
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
# animal = 'MD594'
# aligned_shape = np.array((47000, 23300)) // 32
# num_section = 433
# animal = 'MD585'
# aligned_shape = np.array((38200, 28000)) // 32
# num_section = 446
animal = 'MD589'
aligned_shape = np.array((43700, 32400)) // 32
num_section = 447

color_filepath = os.path.join('../', 'neuroglancer/contours/json_cache', 'struct_to_color_2.json')
with open(color_filepath, 'r') as json_file:
    colors = json.load(json_file)
colors = {name.upper(): index for name, index in colors.items()}

In [3]:
def get_dense_coordinates(coor_list):
    dense_coor_list = []
    # Shortest distance, x, y

    # for x, y in coor_list:
    for i in range(len(coor_list) - 1):
        x, y = coor_list[i]
        x_next, y_next = coor_list[i + 1]

        x_mid = (x + x_next) / 2
        y_mid = (y + y_next) / 2

        dense_coor_list.append([x, y])
        dense_coor_list.append([x_mid, y_mid])

        if i == len(coor_list) - 2:
            dense_coor_list.append([x_next, y_next])
            x, y = coor_list[0]
            x_mid = (x + x_next) / 2
            y_mid = (y + y_next) / 2
            dense_coor_list.append([x_mid, y_mid])

    return dense_coor_list

def get_contours_from_annotations(stack, target_structure, hand_annotations, densify=0):
    MD585_ng_section_min = 83
    num_annotations = len(hand_annotations)
    str_contours_annotation = {}

    for i in range(num_annotations):
        structure = hand_annotations['name'][i]
        side = hand_annotations['side'][i]
        section = hand_annotations['section'][i]
        first_sec = 0
        last_sec = 0

        #if side == 'R' or side == 'L':
        #    structure = structure + '_' + side

        if structure == target_structure:
            vertices = hand_annotations['vertices'][i]

            for i in range(densify):
                vertices = get_dense_coordinates(vertices)

            # Skip sections before the 22nd prep2 section for MD585 as there are clear errors
            if stack == 'MD585' and section < MD585_ng_section_min + 22:
                # vertices = vertices - np.array(MD585_abberation_correction)
                continue
            str_contours_annotation[section] = {}
            str_contours_annotation[section][structure] = {}
            str_contours_annotation[section][structure][1] = vertices

    try:
        first_sec = np.min(list(str_contours_annotation.keys()))
        last_sec = np.max(list(str_contours_annotation.keys()))
    except:
        pass
    return str_contours_annotation, first_sec, last_sec

CSV_PATH = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/foundation_brain_annotations'
csvfile = os.path.join(CSV_PATH, f'{animal}_annotation.csv')
hand_annotations = pd.read_csv(csvfile)
hand_annotations['vertices'] = hand_annotations['vertices'] \
    .apply(lambda x: x.replace(' ', ','))\
    .apply(lambda x: x.replace('\n',','))\
    .apply(lambda x: x.replace(',]',']'))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ',')).apply(lambda x: x.replace(',,', ','))
hand_annotations['vertices'] = hand_annotations['vertices'].apply(lambda x: ast.literal_eval(x))

structures = list(hand_annotations['name'].unique())
section_structure_vertices = defaultdict(dict)
for structure in tqdm(structures):
    contour_annotations, first_sec, last_sec = get_contours_from_annotations(animal, structure, hand_annotations, densify=4)
    for section in contour_annotations:
        section_structure_vertices[section][structure] = contour_annotations[section][structure][1]

100%|██████████| 65/65 [00:03<00:00, 18.69it/s]


In [4]:
all_structures = set()
for section in section_structure_vertices:
    for structure in section_structure_vertices[section].keys():
        all_structures.add(structure)
all_structures = sorted(list(all_structures))

In [5]:
table = []
for section in section_structure_vertices:
    section_structures = list(section_structure_vertices[section].keys())
    
    row = [section]
    for structure in all_structures:
        row.append(structure in section_structures)
    table.append(row)
table = pd.DataFrame(table, columns=['section'] + all_structures)
table = table.sort_values('section')

In [6]:
table.to_excel(f'{animal}_structures.xlsx')